# Anomaly detection ANN
That model based on an Artificial Neural Network (ANN) and using k-fold cross-validation for model fitting

## Imports

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

## Data preparation

### Run for ECG

In [ ]:
dataframe = pd.read_csv('../dataset/ecg_prod_10k.csv', header=None)
raw_data = dataframe.values

labels = raw_data[:, -1]
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=50)

train_data[0]

### Run for signal

In [ ]:
dataframe = pd.read_csv('../dataset/signal_10k.csv', header=None)
raw_data = dataframe.values

labels = raw_data[:, -1]
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=50)


train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

## ANN model

In [ ]:
def build_model(neurons):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(neurons[0], activation='relu', input_shape=(train_data.shape[1],)))
    model.add(tf.keras.layers.Dense(neurons[1], activation='relu'))
    model.add(tf.keras.layers.Dense(neurons[2], activation='relu'))
    model.add(tf.keras.layers.Dense(neurons[3], activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Lion(learning_rate=0.0002, beta_1=0.5),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=[tf.keras.metrics.binary_accuracy]
    )
    return model


## Model training
Below there is a process of fitting model using k-fold cross-validation

In [ ]:
k = int(4)
num_val_samples = len(train_data) // k
epochs = 25
all_scores = []
neurons = [
    [32, 32, 16],
    [128, 32, 16],
    [64,16,8],
    [256, 128, 32]
]
for i in range(k):
    print(f'processing fold #{i}')
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_labels = train_labels[i*num_val_samples: (i+1) * num_val_samples]

    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i+1) * num_val_samples:]],
        axis=0
    )

    partial_train_labels = np.concatenate(
        [train_labels[:i * num_val_samples],
        train_labels[(i+1) * num_val_samples:]],
        axis=0
    )

    model = build_model(neurons[i])

    model.fit(
        partial_train_data,
        partial_train_labels,
        epochs=epochs,
        batch_size=128
    )

    results = model.evaluate(val_data, val_labels, verbose=0)

## Model saving

Fit the best model from tested ones

In [ ]:
model = build_model([32, 32, 16, 8])

model.fit(
    train_data,
    train_labels,
    epochs=25,
    batch_size=128
)

In [ ]:
dataframe = pd.read_csv('../dataset/signal_10k_val.csv', header=None)
random_dataframe = dataframe
raw_data = random_dataframe.values

labels = raw_data[:, -1]
data = raw_data[:, 0:-1]
result = model.predict(data)



correctly_recognized_samples_normal = 0
correctly_recognized_samples_anomalous = 0
temp = []
corr_max = [0, float('inf')]
for i in np.arange(0.01, 1.0, 0.01):
    for j in range(0, len(result)):
        if result[j] < i and labels[j] == 0:
            correctly_recognized_samples_anomalous += 1
        if result[j] > i and labels[j] == 1:
            correctly_recognized_samples_normal += 1
    if abs(correctly_recognized_samples_normal / 5000 - correctly_recognized_samples_anomalous / 5000) < corr_max[1]:
        corr_max = [i,
                    (correctly_recognized_samples_normal / 5000 + correctly_recognized_samples_anomalous / 5000) / 2]
        temp = [correctly_recognized_samples_normal, correctly_recognized_samples_anomalous]
    correctly_recognized_samples_normal = 0
    correctly_recognized_samples_anomalous = 0
print(corr_max)

In [ ]:
print(temp)

In [ ]:
model.save('../models/detectors/ann_signal')

## Model evaluating and testing